## Imports
---
Before we begin, we import the top-level necessary packages. Moreover, we include the algorithm specific packages from the local directories.

In [ ]:
# Get the system environment variables
import os
import pickle
import sys

from IPython import display


# General Imports
import numpy as np
from time import time

# Variational
from auxiliary import collect_obs
from auxiliary import initialize_Ab0
from core import smoothing

# Plotting
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Dynamic
from dynamics import sys_lorenz_63 as dynamics

%matplotlib inline

# Change the default figure size.
plt.rcParams['figure.figsize'] = (10.0, 8.0)

## Grid search for Lorenz 63

In [ ]:
def get_file_name(base_dir, theta_Drift):
    return '{}/{:.2f}-{:.2f}-{:.2f}.pickle'.format(base_dir, theta_Drift[0], theta_Drift[1], theta_Drift[2])

In [ ]:
def get_sample_path_file_name(base_dir):
    return '{}/sample.pickle'.format(base_dir)

In [ ]:
def generate_sample_path(base_dir):
    '''
    TIME-WINDOW PARAMETERS:
    '''
    # Initial, final and time step.
    t0 = 0
    tf = 20
    dt = 0.01

    # Define the time-window of inference.
    Tw = np.arange(t0, tf + dt, dt)

    # Number of discretized points.
    N = Tw.shape[0]

    '''
    SYSTEM SPECIFIC PARAMETERS:
    '''
    # Dimensionality of the system.
    D = 3

    # Stochastic Noise (variance).
    sigma_Noise = 10 * np.eye(D)

    # Drift parameters (sigma, rho, beta).
    theta_Drift = np.array([10, 28, 8. / 3])

    # Observation Noise (variance).
    obs_Noise = 2 * np.eye(D)

    # Define the observation density (# of observations per time unit).
    n_Obs = 5

    # We need at least one observation (per time unit)
    n_Obs = np.max([n_Obs, 1])

    # Observation operator: np.eye(D)
    H = np.eye(D)
    
    # Create the (artificial) true trajectory.
    xt_true = dynamics.system_path(Tw, sigma_Noise, theta_Drift)

    # Sample the noisy observations from the true path.
    obsX, obsY = collect_obs.collect_obs(xt_true, Tw, n_Obs, obs_Noise)
    
    data = {
        't0': t0,
        'tf': tf,
        'dt': dt,
        'Tw': Tw,
        'N': N,
        'D': D,
        'sigma_Noise': sigma_Noise,
        'theta_Drift': theta_Drift,
        'obs_Noise': obs_Noise,
        'n_Obs': n_Obs,
        'H': H,
        'xt_true': xt_true,
        'obsX': obsX,
        'obsY': obsY
    }
    
    file_name = get_sample_path_file_name(base_dir)
    
    with open(file_name, 'wb') as file:
        pickle.dump(data, file)
        
    return data

In [ ]:
def load_sample_path(base_dir):
    file_name = get_sample_path_file_name(base_dir)
    with open(file_name, 'rb') as file:
        return pickle.load(file)

In [ ]:
def run_grid_search(base_dir, data):
    '''
    PRIOR MOMEMTS:
    '''
    # Prior moment of initial condition noise variance.
    # p(x0) ~ N(mu, tau0)
    tau0 = 0.5 * np.eye(data['D'])

    # Get the true sample value at time t=0
    prior_x0 = {'mu0': data['xt_true'][0], 'tau0': tau0}

    # Initial mean m(t=0)
    m0 = data['xt_true'][0] + 0.1 * np.random.randn(1)

    # Initial covariance matrix S(t=0): K*np.eye(D)
    S0 = 0.25 * np.eye(data['D'])


    '''
    PACKING PARAMETERS:
    '''
    # ODE solver: {'Euler', 'Heun', 'RK2', 'RK4'}
    ode_method = 'Euler'

    # Create a dictionary to hold all the model parameters.
    sde_struct = {
        'Sig': data['sigma_Noise'], 
        'theta': data['theta_Drift'], 
        'Rig': data['obs_Noise'],
        'D': data['D'], 
        'H': data['H'], 
        'obsX': data['obsX'],
        'obsY': data['obsY'],
        'px0': prior_x0,
        'Tw': data['Tw'], 
        'dt': data['dt'], 
        'N': data['N'], 
        'ode_method': ode_method,
        'checkGradf': False
    }

    '''
    INITIALIZATION:
    '''
    # Maximum number of iterations.
    nit = 500

    # Generate initial variational parameters (initial search point).
    Ab0 = initialize_Ab0.initialize_Ab0(S0, sde_struct)

    # Main Operation.
    print(' [VGPA] (Smoothing) Experiment in progress. Please wait ...')

    # Start the timer.
    tic = time()

    # Full Variational approximation.
    Fmin, mParam = smoothing.smoothing(dynamics.energy_mode, Ab0, m0, S0, sde_struct, nit)

    # Stop the timer.
    ttime = time() - tic
        
    result = {
        'theta_Drift': data['theta_Drift'],
        'Fmin': Fmin,
        'mParam': mParam,
        'ttime': ttime,
    }

    file_name = get_file_name(base_dir, data['theta_Drift'])
    with open(file_name, 'wb') as file:
        pickle.dump(result, file)

In [ ]:
sigmas = [6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16]
rhos = [24, 24.5, 25, 25.5, 26, 26.5, 27, 27.5, 28, 28.5, 29, 29.5, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34]
betas = [1, 1.5, 2, 2.5, 8. / 3, 3, 3.5, 4]

base_dir = './results/grid-lorenz-63/'

theta_Drift_true = np.array([10, 28, 8. / 3])

if not os.path.isfile(get_sample_path_file_name(base_dir)):
    print('Generating new sample path')
    generate_sample_path(base_dir)    

test_data = load_sample_path(base_dir)
run_grid_search(base_dir, test_data)

for sigma in sigmas:
    for rho in rhos:
        for beta in betas:
            theta_Drift_test = np.array([sigma, rho, beta])
            if os.path.isfile(get_file_name(base_dir, theta_Drift_test)):
                print('Skipped {}'.format(theta_Drift_test))
            else:
                print('Running for {}'.format(theta_Drift_test))
                test_data = load_sample_path(base_dir)            
                test_data['theta_Drift'] = theta_Drift_test
                run_grid_search(base_dir, test_data)
                display.clear_output()                 

In [ ]:
def load_surface_data(base_dir, sigmas, rhos, bestas):
    surface_data = []
    
    for sigma in sigmas:
        for rho in rhos:
            for beta in betas:
                file_name = get_file_name(base_dir, np.array([sigma, rho, beta]))
                if not os.path.isfile(file_name):
                    continue
                    
                with open(file_name, 'rb') as file:
                    datum = pickle.load(file)
                    surface_data.append([sigma, rho, beta, datum['Fmin']])
    surface_data = np.array(surface_data)
    return surface_data

In [ ]:
def plot_grid_search_result(data, x, x_range, y, y_range, f, f_level):    
    fig = plt.figure(figsize=plt.figaspect(0.8))
    ax = fig.add_subplot(1, 1, 1, projection='3d')

    labels = ['σ', 'ρ', 'ß']
    ax.set_xlabel(labels[x])
    ax.set_ylabel(labels[y])
    ax.set_zlabel('Free energy ({}={:.2f})'.format(labels[f], f_level))

    X = []
    Y = []
    Z = []

    for i in range(data.shape[0]):
        if data[i, f] == f_level:
            X.append(data[i, x])
            Y.append(data[i, y])
            Z.append(data[i, -1])
            
    X, Y = np.meshgrid(x_range, y_range)
    Z = np.array(Z).reshape(X.shape)
    surf = ax.plot_surface(X, Y, Z, linewidth=1, alpha=0.95, antialiased=True, 
                           rstride=1, cstride=1, cmap=cm.bone)
    ax.view_init(elev=30, azim=-40)
    plt.show()    
    plt.savefig('{}-{}-{}.png'.format(x, y, f), dpi=270, bbox_inches='tight', pad_inches=0.2)

In [ ]:
sigmas = [6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16]
rhos = [24, 24.5, 25, 25.5, 26, 26.5, 27, 27.5, 28, 28.5, 29, 29.5, 30, 30.5, 31, 31.5, 32, 32.5, 33, 33.5, 34]
betas = [1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4]

base_dir = './results/grid-lorenz-63/'

%matplotlib notebook
plt.rcParams['figure.figsize'] = (10.0, 8.0)

surface_data = load_surface_data(base_dir, sigmas, rhos, betas)

In [ ]:
plot_grid_search_result(surface_data, 0, sigmas, 1, rhos, 2, 2.5)
plot_grid_search_result(surface_data, 0, sigmas, 2, betas, 1, 28)
plot_grid_search_result(surface_data, 1, rhos, 2, betas, 0, 10)